In [1]:
!pip install pyathena
!pip install pyarrow boto3

In [2]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import os

from pyathena import connect
from pyathena.pandas.util import as_pandas

import io
import pyarrow.parquet as pq
import ast

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
cursor = connect(s3_staging_dir="s3://aws-athena-query-results-185381186363-us-west-2", region_name='us-west-2').cursor()

In [19]:
def list_files(s3_client, bucket, prefix, pattern):
    """
    Generator that traverses through an S3 bucket and lists all files that match the given pattern
    """
    paginator = s3_client.get_paginator('list_objects_v2')

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        if 'Contents' in page:
            for obj in page['Contents']:
                if pattern in obj['Key'].lower():
                    yield obj['Key']


def download_file(s3_client, bucket, key, local_dir):
    """
    Download a file from S3 to the local file system
    """
    filename = os.path.basename(key)
    local_path = os.path.join(local_dir, filename)
    
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    
    s3_client.download_file(bucket, key, local_path)     
    
    
def copy_file_within_s3(s3_client, source_bucket, source_key, target_bucket, target_prefix):
    """
    Copy a file from one S3 location to another S3 location
    """
    filename = os.path.basename(source_key)
    target_key = os.path.join(target_prefix, filename)
    
    copy_source = {
        'Bucket': source_bucket,
        'Key': source_key
    }
    
    s3_client.copy(copy_source, target_bucket, target_key)
    print(f"Copied {source_key} from {source_bucket} to {target_key} in {target_bucket}")


In [ ]:
# ##all files related to the company in s3
# for key in list_files(s3_client, s3_bucket, folder_path, 'mosanna'):
#     print(key)

In [ ]:
# s3_client = boto3.client('s3', region_name='us-west-2')

# source_bucket = 'plfs-data-prod-us-west-2'
# source_prefix = 'sevanta/files_raw/nfls/_Deals'
# target_bucket = 'plfs-han-llm-experiment'
# target_prefix = f"2pager/{COMPANY_NAME}/internal/jarvis_docs"
# pattern = company

# for key in list_files(s3_client, source_bucket, source_prefix, pattern):
#     copy_file_within_s3(s3_client, source_bucket, key, target_bucket, target_prefix)

In [ ]:
def delete_files_from_s3(bucket_name, keys):
    """
    Delete a list of files from an S3 bucket.

    Parameters:
    bucket_name (str): The name of the S3 bucket.
    keys (list): A list of keys (file paths) to delete.
    """
    s3_client = boto3.client('s3')

    # Prepare the list of objects to delete
    objects_to_delete = [{'Key': key} for key in keys]

    # Delete the objects
    response = s3_client.delete_objects(
        Bucket=bucket_name,
        Delete={
            'Objects': objects_to_delete,
            'Quiet': True
        }
    )

    if 'Errors' in response:
        print("Errors occurred while deleting the following keys:")
        for error in response['Errors']:
            print(f"Key: {error['Key']}, Error: {error['Message']}")
    else:
        print("Files deleted successfully.")


In [ ]:
# ##delete some giant and useless files
# bucket_name = 'plfs-han-llm-experiment'
# prefix = 'ic-memo-generation/Mosanna/docs_ic_memo/'

# keys_to_delete = [f"{prefix}3.1.2.2 Preformulation Report AVE0118.pdf",
#                   f"{prefix}5.1.2 2.2 US9056833 patent file history USPTO download.pdf"
#                  ]

# delete_files_from_s3(bucket_name, keys_to_delete)

### unzip .zip file in s3

In [3]:
import zipfile
import io

def unzip_s3_file(bucket_name, zip_key, destination_prefix):
    # Get the zip file object from S3
    zip_obj = s3.get_object(Bucket=bucket_name, Key=zip_key)
    buffer = io.BytesIO(zip_obj['Body'].read())
    
    # Unzip the file contents
    with zipfile.ZipFile(buffer, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            # Get the filename and decode it properly
            # Try to decode with UTF-8 first, then fall back to CP437 or GB2312/GBK for Chinese
            filename = file_info.filename
            try:
                # Try to encode as CP437 and decode as UTF-8 (common for files zipped on Windows)
                filename = file_info.filename.encode('cp437').decode('utf-8')
            except (UnicodeDecodeError, UnicodeEncodeError):
                try:
                    # Try GB2312/GBK encoding (common for Chinese Windows systems)
                    filename = file_info.filename.encode('cp437').decode('gbk')
                except (UnicodeDecodeError, UnicodeEncodeError):
                    # If all else fails, keep the original filename
                    pass
            
            # Skip .DS_Store files and __MACOSX folder
            if filename.endswith('.DS_Store') or filename.startswith('__MACOSX/'):
                continue
            
            if not file_info.is_dir():
                # Upload each file to S3 with the corrected filename
                with zip_ref.open(file_info) as file_data:
                    s3.upload_fileobj(
                        file_data, 
                        Bucket=bucket_name, 
                        Key=f"{destination_prefix}{filename}"
                    )

In [4]:
s3 = boto3.client('s3')
s3_bucket_name = 'plfs-han-llm-experiment'
zip_key = "factsheet-generation/PPInnova/PPInnova_sharepoint_selected.zip"  # The S3 key of the zip file
destination_prefix = "factsheet-generation/PPInnova/"  # The S3 prefix where unzipped files will be stored

unzip_s3_file(s3_bucket_name, zip_key, destination_prefix)

In [12]:
s3 = boto3.client('s3')
s3_bucket_name = 'plfs-han-llm-experiment'
zip_key = "ic-memo-generation/Trevi/trvi_docs.zip"  # The S3 key of the zip file
destination_prefix = "ic-memo-generation/Trevi/Trevi_sharepoint_selected/"  # The S3 prefix where unzipped files will be stored

unzip_s3_file(s3_bucket_name, zip_key, destination_prefix)

In [ ]:
# s3 = boto3.client('s3')
# s3_bucket_name = 'plfs-han-llm-experiment'
# zip_key = 'ic-memo-generation/Geneditor/Key_LLM_Documents.zip'  # The S3 key of the zip file
# destination_prefix = 'ic-memo-generation/Geneditor/'  # The S3 prefix where unzipped files will be stored

# unzip_s3_file(s3_bucket_name, zip_key, destination_prefix)

In [5]:
##copy files in a s3 folder to another s3 folder, do not keep the original folder structure if subfolder exist,
##and rename the file to add its highest subfolder's name such as 'Corporate decks/','Dataroom/'

def copy_files_flattened_and_renamed_within_s3(bucket_name, source_prefix, destination_prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=source_prefix)
    if 'Contents' not in response:
        print(f"No files found in {source_prefix}")
        return

    for obj in response['Contents']:
        source_key = obj['Key']
        if source_key.endswith('/'):
            continue  # Skip directories

        # Extract the relative path from source_prefix
        relative_path = os.path.relpath(source_key, source_prefix)
        
        # Extract the top folder name
        top_folder_name = relative_path.split('/')[0]
        
        # Extract the original file name
        original_file_name = os.path.basename(source_key)
        
        # Create the new file name with the folder name included
        new_file_name = f"{top_folder_name}_{original_file_name}"
        
        # Construct the destination key
        destination_key = f"{destination_prefix.rstrip('/')}/{new_file_name}"
        
        # Copy the object to the new location
        copy_source = {'Bucket': bucket_name, 'Key': source_key}
        s3.copy_object(CopySource=copy_source, Bucket=bucket_name, Key=destination_key)
        
        print(f"Copied {source_key} to {destination_key}")


In [ ]:
# s3 = boto3.client('s3')

# bucket_name = 'plfs-han-llm-experiment'
# source_prefix = 'ic-memo-generation/Geneditor/Geneditor_sharepoint_selected/'  # The S3 prefix where unzipped files are stored
# destination_prefix = 'ic-memo-generation/Geneditor/sharepoint_flattened_files/'  # The S3 prefix where files will be copied

# copy_files_flattened_and_renamed_within_s3(bucket_name, source_prefix, destination_prefix)


In [6]:
##copy files in a s3 folder to another s3 folder, do not keep the original folder structure if subfolder exist,
##and rename the file to add its folder structure, which will transform paths like A/B/C/filename.pdf into A--B--C--filename.pdf

def copy_files_flattened_and_renamed_within_s3_version2(bucket_name, source_prefix, destination_prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=source_prefix)
    if 'Contents' not in response:
        print(f"No files found in {source_prefix}")
        return

    # Extract the last folder name from source_prefix
    last_folder_name = os.path.basename(source_prefix.rstrip('/'))

    for obj in response['Contents']:
        source_key = obj['Key']
        if source_key.endswith('/'):
            continue  # Skip directories

        # Extract the relative path from source_prefix
        relative_path = os.path.relpath(source_key, source_prefix)
        
        # Replace all '/' in the relative path with '--' to flatten the path
        flattened_path = relative_path.replace('/', '--')
        
        # Construct the new file name by including the last folder name from source_prefix
        new_file_name = f"{last_folder_name}--{flattened_path}"
        
        # Construct the destination key
        destination_key = f"{destination_prefix.rstrip('/')}/{new_file_name}"
        
        # Copy the object to the new location
        copy_source = {'Bucket': bucket_name, 'Key': source_key}
        s3.copy_object(CopySource=copy_source, Bucket=bucket_name, Key=destination_key)
        
        print(f"Copied {source_key} to {destination_key}")


In [7]:
s3 = boto3.client('s3')

bucket_name = 'plfs-han-llm-experiment'
source_prefix = 'factsheet-generation/PPInnova/PPInnova_sharepoint_selected/'  # The S3 prefix where unzipped files are stored
destination_prefix = 'factsheet-generation/PPInnova/sharepoint_flattened_files/'  # The S3 prefix where files will be copied

copy_files_flattened_and_renamed_within_s3_version2(bucket_name, source_prefix, destination_prefix)


Copied factsheet-generation/PPInnova/PPInnova_sharepoint_selected/20250507-Pivotal Bioventure Question list for PP Innova - RBM3920250509v2.pdf to factsheet-generation/PPInnova/sharepoint_flattened_files/PPInnova_sharepoint_selected--20250507-Pivotal Bioventure Question list for PP Innova - RBM3920250509v2.pdf
Copied factsheet-generation/PPInnova/PPInnova_sharepoint_selected/20250801-Project+Progress+Update+STAT6&+TNFSF+Pivotal碧沃投 (1).pdf to factsheet-generation/PPInnova/sharepoint_flattened_files/PPInnova_sharepoint_selected--20250801-Project+Progress+Update+STAT6&+TNFSF+Pivotal碧沃投 (1).pdf
Copied factsheet-generation/PPInnova/PPInnova_sharepoint_selected/250522_PP Innova Opportunity Discussion_vPresentation.pdf to factsheet-generation/PPInnova/sharepoint_flattened_files/PPInnova_sharepoint_selected--250522_PP Innova Opportunity Discussion_vPresentation.pdf
Copied factsheet-generation/PPInnova/PPInnova_sharepoint_selected/250703_PP Innova Fact Sheet (deal update).pdf to factsheet-gener

In [24]:
s3 = boto3.client('s3')

bucket_name = 'plfs-han-llm-experiment'
source_prefix = 'ic-memo-generation/RiboX/RiboX_sharepoint_selected/'  # The S3 prefix where unzipped files are stored
destination_prefix = 'ic-memo-generation/RiboX/sharepoint_flattened_files/'  # The S3 prefix where files will be copied

copy_files_flattened_and_renamed_within_s3_version2(bucket_name, source_prefix, destination_prefix)


Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/1. NDA - NFIM - RiboX_250710_signed.pdf to ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--1. NDA - NFIM - RiboX_250710_signed.pdf
Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/1_RiboX_Team Member research_20250709.xlsx to ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--1_RiboX_Team Member research_20250709.xlsx
Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/1_RiboX_Team Research_250707.docx to ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--1_RiboX_Team Research_250707.docx
Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/2_circRNA vs mRNA_ Industry Analysis_20250708.docx to ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--2_circRNA vs mRNA_ Industry Analysis_20250708.docx
Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/CircRNA/202504 σ£åσ¢áτöƒτë⌐BP.pdf to ic-memo-g

Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/RiboX Corporate Overview_ 202505_nonconfidential.pdf to ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--RiboX Corporate Overview_ 202505_nonconfidential.pdf
Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/RiboX-Weiyi Zhang-20250710.m4a to ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--RiboX-Weiyi Zhang-20250710.m4a
Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/RiboX-meetingnotes-Hongbo Lu-250629.pdf to ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--RiboX-meetingnotes-Hongbo Lu-250629.pdf
Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/RiboX_CEO_WeiyiZhang_Notes_20250710.docx to ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--RiboX_CEO_WeiyiZhang_Notes_20250710.docx
Copied ic-memo-generation/RiboX/RiboX_sharepoint_selected/RiboX_Weiyi Zhang_20240710.mp4 to ic-memo-generation

In [8]:
def copy_files_from_s3_to_s3(source_bucket: str, source_prefix: str, destination_bucket: str, destination_prefix: str):
    """
    Copies all files from source_prefix in source_bucket to destination_prefix in destination_bucket.
    """
    s3_client = boto3.client('s3')

    # List all objects in the source directory
    response = s3_client.list_objects_v2(Bucket=source_bucket, Prefix=source_prefix)
    if 'Contents' not in response:
        print(f"No files found in {source_bucket}/{source_prefix}")
        return

    for obj in response['Contents']:
        source_key = obj['Key']
        destination_key = source_key.replace(source_prefix, destination_prefix, 1)
        
        copy_source = {
            'Bucket': source_bucket,
            'Key': source_key
        }
        
        print(f"Copying {source_bucket}/{source_key} to {destination_bucket}/{destination_key}")
        s3_client.copy(copy_source, destination_bucket, destination_key)

    print(f"Copied files from s3://{source_bucket}/{source_prefix} to s3://{destination_bucket}/{destination_prefix}")



In [9]:
s3_bucket = 'plfs-han-llm-experiment'
dir1 = 'factsheet-generation/PPInnova/internal/jarvis_docs/'
dir2 = 'factsheet-generation/PPInnova/sharepoint_flattened_files/'

copy_files_from_s3_to_s3(s3_bucket, dir2, s3_bucket, dir1)

Copying plfs-han-llm-experiment/factsheet-generation/PPInnova/sharepoint_flattened_files/PPInnova_sharepoint_selected--20250507-Pivotal Bioventure Question list for PP Innova - RBM3920250509v2.pdf to plfs-han-llm-experiment/factsheet-generation/PPInnova/internal/jarvis_docs/PPInnova_sharepoint_selected--20250507-Pivotal Bioventure Question list for PP Innova - RBM3920250509v2.pdf
Copying plfs-han-llm-experiment/factsheet-generation/PPInnova/sharepoint_flattened_files/PPInnova_sharepoint_selected--20250801-Project+Progress+Update+STAT6&+TNFSF+Pivotal碧沃投 (1).pdf to plfs-han-llm-experiment/factsheet-generation/PPInnova/internal/jarvis_docs/PPInnova_sharepoint_selected--20250801-Project+Progress+Update+STAT6&+TNFSF+Pivotal碧沃投 (1).pdf
Copying plfs-han-llm-experiment/factsheet-generation/PPInnova/sharepoint_flattened_files/PPInnova_sharepoint_selected--250522_PP Innova Opportunity Discussion_vPresentation.pdf to plfs-han-llm-experiment/factsheet-generation/PPInnova/internal/jarvis_docs/PPInn

In [26]:
s3_bucket = 'plfs-han-llm-experiment'
dir1 = 'ic-memo-generation/RiboX/internal/jarvis_docs/'
dir2 = 'ic-memo-generation/RiboX/sharepoint_flattened_files/'

copy_files_from_s3_to_s3(s3_bucket, dir2, s3_bucket, dir1)

Copying plfs-han-llm-experiment/ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--1. NDA - NFIM - RiboX_250710_signed.pdf to plfs-han-llm-experiment/ic-memo-generation/RiboX/internal/jarvis_docs/RiboX_sharepoint_selected--1. NDA - NFIM - RiboX_250710_signed.pdf
Copying plfs-han-llm-experiment/ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--1_RiboX_Team Member research_20250709.xlsx to plfs-han-llm-experiment/ic-memo-generation/RiboX/internal/jarvis_docs/RiboX_sharepoint_selected--1_RiboX_Team Member research_20250709.xlsx
Copying plfs-han-llm-experiment/ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--1_RiboX_Team Research_250707.docx to plfs-han-llm-experiment/ic-memo-generation/RiboX/internal/jarvis_docs/RiboX_sharepoint_selected--1_RiboX_Team Research_250707.docx
Copying plfs-han-llm-experiment/ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--2_circRNA vs mRNA_ Ind

Copying plfs-han-llm-experiment/ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--Literature Reviews & Patent--CN113101363AΣ╕ÇτºìτÄ»τè╢RNAτû½ΦïùσÅèσà╢σ║öτö¿-20250708.pdf to plfs-han-llm-experiment/ic-memo-generation/RiboX/internal/jarvis_docs/RiboX_sharepoint_selected--Literature Reviews & Patent--CN113101363AΣ╕ÇτºìτÄ»τè╢RNAτû½ΦïùσÅèσà╢σ║öτö¿-20250708.pdf
Copying plfs-han-llm-experiment/ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--Literature Reviews & Patent--Circular RNA- an emerging frontier in RNA therapeutic targets, RNA therapeutics, and mRNA vaccines_20250708.pdf to plfs-han-llm-experiment/ic-memo-generation/RiboX/internal/jarvis_docs/RiboX_sharepoint_selected--Literature Reviews & Patent--Circular RNA- an emerging frontier in RNA therapeutic targets, RNA therapeutics, and mRNA vaccines_20250708.pdf
Copying plfs-han-llm-experiment/ic-memo-generation/RiboX/sharepoint_flattened_files/RiboX_sharepoint_selected--Literatur

In [ ]:
# ##delete some giant and useless files
# bucket_name = 'plfs-han-llm-experiment'
# prefix = 'ic-memo-generation/Mosanna/docs_ic_memo_new/'

# keys_to_delete = [f"{prefix}3.1.2.2 Preformulation Report AVE0118.pdf",
#                   f"{prefix}5.1.2 2.2 US9056833 patent file history USPTO download.pdf"
#                  ]

# delete_files_from_s3(bucket_name, keys_to_delete)